In [1]:
import pandas as pd
import numpy as np
import xarray as xr
# import cartopy.crs as ccrs
# import cartopy.feature as cfeat
# import matplotlib.pyplot as plt
# import matplotlib.animation as animation
import geopandas as gpd
import rioxarray
from shapely.geometry import mapping

In [2]:
def centroids(ds):
    max_precip=ds.max(dim="time",skipna=True)
    min_precip=ds.min(dim="time",skipna=True)
    a = (ds - min_precip+0.0000001)/(max_precip - min_precip+0.0000001)
    y = a.lat.dot(a.precip.fillna(0) , dims=["lon","lat"])/ a.precip.fillna(0).sum(skipna=True , dim=["lat","lon"])
    x = a.lon.dot(a.precip.fillna(0) , dims=["lon","lat"])/ a.precip.fillna(0).sum(skipna=True , dim=["lat","lon"])
    return (x,y)


In [3]:
ds = xr.open_mfdataset("/mnt/d/UGP/datasets/full_data_daily_v2020_10_2000.nc").sel(lat = slice(5,45) , lon = slice(65,100))
ds = ds.precip.rio.set_crs(4326)
sf = gpd.read_file('/mnt/d/UGP/datasets/hydrosheds_India.shp')

In [9]:
clipped = ds.rio.clip(sf.geometry.apply(mapping), sf.crs)
clipped

<xarray.DataArray 'precip' (time: 366, lat: 34, lon: 35)>
dask.array<copy, shape=(366, 34, 35), dtype=float32, chunksize=(366, 34, 35), chunktype=numpy.ndarray>
Coordinates:
  * lat          (lat) float64 6.5 7.5 8.5 9.5 10.5 ... 35.5 36.5 37.5 38.5 39.5
  * lon          (lon) float64 65.5 66.5 67.5 68.5 69.5 ... 96.5 97.5 98.5 99.5
  * time         (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-12-31
    spatial_ref  int64 0
Attributes:
    long_name:  gpcc full data daily product version 2020 precipitation per grid
    units:      mm/day
    code:       20

In [20]:
# ShapeMask = rasterio.features.geometry_mask(sf.iloc[0],out_shape=(len(ds.lat), len(ds.lon)), invert=True)
# ShapeMask = xr.DataArray(ShapeMask , dims=("lat", "lon"))

x,y = centroids(ds)

In [21]:
def make_figure():
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    plt.ylim(5,45)
    plt.xlim(65,100)
    # generate a basemap with country borders, oceans and coastlines
    ax.add_feature(cfeat.LAND)
    ax.add_feature(cfeat.OCEAN)
    ax.add_feature(cfeat.COASTLINE)
    # ax.add_feature(cfeat.BORDERS, linestyle='dotted')
    return fig, ax

In [22]:
for i in range(0,len(x),30):
    _, ax = make_figure()
    plt.scatter(x[i],y[i])
    plt.title("Day:" + str(i+1))
    plt.show()

NameError: name 'plt' is not defined

In [16]:
fig, ax = make_figure()

frames = len(x)

def draw(frame):
    plot = ax.scatter(x[frame],y[frame])
    ax.set_title("Day:" + str(frame+1))
    return plot


def init():
    return draw(0)


def animate(frame):
    return draw(frame)


ani = animation.FuncAnimation(fig, animate, frames, interval=0.01, blit=False,init_func=init, repeat=False)
plt.close(fig)

In [17]:
ani.save('precip.mp4', writer=animation.FFMpegWriter(fps=5))

/home/rghoshal/anaconda3/lib/python3.7/site-packages/dask/compatibility.py:93: RuntimeWarning: All-NaN slice encountered
  return func(*args, **kwargs)
/home/rghoshal/anaconda3/lib/python3.7/site-packages/toolz/functoolz.py:468: RuntimeWarning: All-NaN slice encountered
  ret = f(ret)


In [5]:
sf.dissolve(by="HYBAS_ID")

,geometry,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,SIDE,LAKE,ENDO,COAST,ORDER,SORT
HYBAS_ID,,,,,,,,,,,,,,,
4040017030,"MULTIPOLYGON (((99.98832 13.47982, 99.98688 13...",0,4040017030,4040017030,0.0,0.0,235496.1,235496.1,4430,M,0,0,1,0,76
4040018350,"MULTIPOLYGON (((99.80326 6.16697, 99.80401 6.1...",0,4040018350,4040018350,0.0,0.0,292240.9,292240.9,4440,M,0,0,1,0,77
4040022790,"MULTIPOLYGON (((97.73742 15.17351, 97.73194 15...",0,4040022790,4040022790,0.0,0.0,10642.3,10642.3,4451,M,0,0,1,0,79
4040023050,"POLYGON ((98.64167 15.64167, 98.64252 15.63835...",0,4040023050,4040023050,0.0,0.0,9409.8,9410.0,4452,M,0,0,0,1,80
4040023060,"POLYGON ((100.00000 23.69583, 99.99943 23.7063...",0,4040023060,4040023060,0.0,0.0,266292.4,266292.4,4454,M,0,0,0,1,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041043580,"MULTIPOLYGON (((99.99779 18.90473, 99.98554 18...",4041108500,4040017020,4040017020,2136.1,2136.1,231768.3,231768.3,4429,M,0,0,0,1,74
4049000420,"POLYGON ((100.00000 36.65525, 99.99843 36.6564...",4040578640,4049000420,4040007850,0.0,3464.0,4454.0,29604.7,4328,M,1,2,0,3,189
4049000880,"POLYGON ((90.89126 30.85515, 90.89520 30.85196...",0,4049000880,4049000880,0.0,0.0,1939.3,10741.3,4916,M,1,2,0,1,208
